In [5]:
import pickle
import numpy
import pydot
import networkx as nx
from parse import tokenizer
from gensim.models import Word2Vec, KeyedVectors
from networkx.drawing.nx_pydot import from_pydot
import numpy as np
import json
MAX_NODES=500

after_fn = "/home/ajinkya/TSE/mp1/joern-parse-output/after/graphs.dot"

def get_next_graph(fn):
    graph_lines=[]
    with open(fn) as f:
        while True:
            line = f.readline()
            if line is None: 
                break
#             if line.startswith('digraph'):
#                 line = line.replace(":", "_").replace('~', "_")
            if line == "\n":
                graph = ''.join(graph_lines)
                graph_lines=[]
                yield graph
            else:
                graph_lines.append(line)
model = KeyedVectors.load_word2vec_format('w2v.model.txt', binary=False)


In [6]:
# with open('function_name_to_graph') as f:
#     funtion_name_to_graph = pickle.load(f)

def check_node_with_label_exist(fname, label):
    g = function_name_to_graph[fname]
    #iterate over before veritces
    for v in g[0]:
        if label in v[1]:
            return True
    return False
            

In [12]:

for x in final_dataset:
    if x['vuln']:
        print(x)
        break

{'name': 'nsc_rle_decode', 'astParentFullName': '', 'columnNumberEnd': 0, 'fullName': 'nsc_rle_decode', 'signature': 'static void nsc_rle_decode (BYTE *,BYTE *,UINT32)', 'astParentType': '', 'columnNumber': 0, 'lineNumberEnd': 139, '_label': 'METHOD', 'code': 'nsc_rle_decode (BYTE* in,BYTE* out,UINT32 originalSize)', 'isExternal': False, 'lineNumber': 95, 'id': 2743054800571798583, 'order': -1, 'filename': '/home/ajinkya/TSE/mp1/files/before/nsc.c', 'source': '{\n\tUINT32 len;\n\tUINT32 left;\n\tBYTE value;\n\tleft = originalSize;\n\n\twhile (left > 4)\n\t{\n\t\tvalue = *in++;\n\n\t\tif (left == 5)\n\t\t{\n\t\t\t*out++ = value;\n\t\t\tleft--;\n\t\t}\n\t\telse if (value == *in)\n\t\t{\n\t\t\tin++;\n\n\t\t\tif (*in < 0xFF)\n\t\t\t{\n\t\t\t\tlen = (UINT32) * in++;\n\t\t\t\tlen += 2;\n\t\t\t}\n\t\t\telse\n\t\t\t{\n\t\t\t\tin++;\n\t\t\t\tlen = *((UINT32*) in);\n\t\t\t\tin += 4;\n\t\t\t}\n\n\t\t\tFillMemory(out, len, value);\n\t\t\tout += len;\n\t\t\tleft -= len;\n\t\t}\n\t\telse\n\t\t{\n\t\

In [7]:
with open('final-dataset.json') as f:
    final_dataset = json.load(f)

In [ ]:
for graph in get_next_graph(after_fn):
    errored_graphs = []
function_name_to_graph = {}
for g in get_next_graph(before_fn):
#     print(g)
    try:
        pgg = pydot.graph_from_dot_data(g)
    except Exception as e:
        print('Errored', g.split('\n')[0])
        errored_graphs.append((g, e))
        continue
    for pg in pgg:
        ng = nx.nx_pydot.from_pydot(pg)
        print('Current Function:', ng.name)
        vertices = fuction_name_to_graph[ng.name][0]
        new_vertices={}
        idx = len(vertices.keys())
        all_new_labs = []
        for k,v in ng.nodes.items():
            lab = v['label'][:-2].split(',')[-1]
            all_new_labs.append(lab)
            if not check_node_with_label_exists(ng.name, lab):
                toks = tokenizer(lab)
                vecs = [model[t] for t in toks if t in model.vocab]
                node_embedding = np.mean(np.asarray(vecs), axis=0)
                vertices[k] = (idx, lab, node_embedding)
                new_vertices[k] = (idx, lab, node_embedding)
                idx+=1
        deleted_vertices = []
        for v in vertices:
            if v[1] not in all_new_labs:
                deleted_vertices.append(v)
#         print(vertices)
        nv = MAX_NODES
        patch_add = np.zeros(shape=(nv,nv))
        patch_sub = np.zeros(shape=(nv,nv))
        for k, v in new_vertices.items():
            for d in deleted_vertices():
                replaced=False
                for h in final_dataset['patch']:
                    if v[1] in h.source and d[1] in h.source:
                        patch_add[v[0], d[0]] +=1
                        patch_add[d[0], v[0]] +=1
                        replace=True
            if not replaced:
                patch_sub[d[0],MAX_NODE-1] += 1
        ast_adj = function_name_to_graph[ng.name][1]
        cfg_adj = function_name_to_graph[ng.name][2]
        cdg_adj = function_name_to_graph[ng.name][3] 
        function_name_to_graph[ng.name] = (vertices, ast_adj, cfg_adj, cdg_adj,patch_add, patch_sub)
